In [1]:
%load_ext autoreload
%autoreload 2

import os
import wget
import zipfile
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from utils import *
from collections import Counter
from tqdm import tqdm
import time
import geopandas as gpd
import gdown # for downloading files from google drive
import shutil
import warnings
import sys
from pyvis.network import Network
warnings.filterwarnings("ignore")

In [2]:
import multiprocessing
import random
import networkx as nx
import numpy as np
import math

def parallel_omega(G, nrand=10, seed=None):

    random.seed(seed)
    if not nx.is_connected(G):
        G = G.subgraph(max(nx.connected_components(G), key=len))

    if len(G) == 1:
        return 0

    niter_lattice_reference = nrand
    niter_random_reference = nrand * 2
    
    def worker(queue):
        while True:
            task = queue.get()
            if task is None:
                break
            random_graph = nx.random_reference(G)
            lattice_graph = nx.lattice_reference(G)
            random_shortest_path = nx.average_shortest_path_length(random_graph)
            lattice_clustering = nx.average_clustering(lattice_graph)
            queue.put((random_shortest_path, lattice_clustering))
               
    n_processes = multiprocessing.cpu_count()
    manager = multiprocessing.Manager()
    queue = manager.Queue()
    processes = [multiprocessing.Process(target=worker, args=(queue,)) for _ in range(n_processes)]
    for process in processes:
        process.start()
    
    for _ in range(nrand):
        queue.put(1)
    
    for _ in range(n_processes):
        queue.put(None)
    
    for process in processes:
        process.join()
    
    shortest_paths = []
    clustering_coeffs = []
    while not queue.empty():
        random_shortest_path, lattice_clustering = queue.get()
        shortest_paths.append(random_shortest_path)
        clustering_coeffs.append(lattice_clustering)
    
    L = nx.average_shortest_path_length(G)
    C = nx.average_clustering(G)

    # kill the process
    for process in processes:
        process.terminate()
        process.join()

    omega = (np.mean(shortest_paths) / L) - (C / np.mean(clustering_coeffs))


    return omega

In [3]:
G = nx.erdos_renyi_graph(200, 0.03)
G = G.subgraph(max(nx.connected_components(G), key=len))
nx.info(G)

'Graph with 200 nodes and 584 edges'

In [5]:
omega = parallel_omega(G, nrand=10, seed=42)
omega

0.6776975801779451

In [4]:
standard_omega = nx.omega(G, nrand=10, seed=42)
standard_omega

KeyboardInterrupt: 